<a href="https://colab.research.google.com/github/srijan-singh/FaceDetect/blob/main/API_Face_Detect_V1_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Dependencies
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install face_recognition
!pip install aiofiles
!pip install python-multipart

In [ ]:
#@title Class Face Detect
import face_recognition
import imageio
import cv2
import numpy as np
import time


class faceDetect:

  def __init__ (self, user_image, user_label):

    face_1 = face_recognition.load_image_file(user_image)
    face_1_encoding = face_recognition.face_encodings(face_1)[0]

    self.known_face_encodings = [face_1_encoding]

    self.known_face_names = [user_label]

  def predict(self, pred_user_image):

    file_name = pred_user_image

    unknown_image = face_recognition.load_image_file(file_name)
    unknown_image_to_draw = cv2.imread(file_name)

    face_locations = face_recognition.face_locations(unknown_image)
    face_encodings = face_recognition.face_encodings(unknown_image, face_locations)

    for (top,right, bottom, left), face_encoding in zip(face_locations, face_encodings):

      matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)

      name = "Unknown"

      face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
      best_match_index = np.argmin(face_distances)
      
      if matches[best_match_index]:
        
        name = self.known_face_names[best_match_index]
      cv2.rectangle(unknown_image_to_draw, (left, top), (right, bottom),(0,255,0),3)
      cv2.putText(unknown_image_to_draw,name, (left, top-20), cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2, cv2.LINE_AA)
  
    time.sleep(10)

    return cv2.imwrite("/content/pred_image.png", unknown_image_to_draw)

In [ ]:
#@title Ngrok Authorization
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok
!/ngrok authtoken YOUR_AUTH_CODE

In [ ]:
#@title API
#Libraries : Only for Colab (To Get Public Link)
import nest_asyncio
from pyngrok import ngrok
from pydantic import BaseModel

# Librarires : Must
import uvicorn
import shutil
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import FileResponse


app = FastAPI()

data_file = "upload_image.png"
target_file = "target_image.png"
data_label = ["Default"]

@app.get("/")
def index():
    return {"Computer Vision":"Face Detect"}


@app.post("/upload/data_file")
async def upload(Source_Image: UploadFile = File(...), Name:str = Form(...)):
    
    data_label.append(Name)

    with open(data_file, "wb") as buffer:
        shutil.copyfileobj(Source_Image.file, buffer)

    return {"filename": Source_Image.filename}


@app.post("/upload/target_file")
async def Target_Image(Target_Image: UploadFile = File(...)):

    with open(target_file, "wb") as buffer:
        shutil.copyfileobj(Target_Image.file, buffer)

    return {"filename": Target_Image.filename}


@app.get("/result")
async def Prediction():
  model = faceDetect(data_file, data_label[-1])
  model.predict(target_file)
  result = FileResponse("/content/pred_image.png")
  return result


if __name__ == "__main__":
  ngrok_tunnel = ngrok.connect(8000)
  print('Public URL:', ngrok_tunnel.public_url)
  nest_asyncio.apply()
  uvicorn.run(app, port=8000)